In [1]:
import theano.tensor as T
from theano import function

In [2]:
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = function([x, y], z)

In [3]:
f(2,3)

array(5.0)

In [4]:
type(x)

theano.tensor.var.TensorVariable

In [5]:
x.type

TensorType(float64, scalar)

In [6]:
from theano import pp

In [7]:
pp(z)

'(x + y)'

In [8]:
x = T.dmatrix('x')
y = T.dmatrix('y')
z = x + y
f = function([x, y], z)

In [9]:
import numpy as np

In [10]:
f(np.array([[1,2],[3,4]]), np.array([[1,2],[3,4]]))

array([[ 2.,  4.],
       [ 6.,  8.]])

### Logistic function

In [11]:
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = function([x], s)
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

### Default arguments

In [12]:
from theano import Param

In [13]:
x, y = T.dscalars('x', 'y')
z = x + y
f = function([x, Param(y, default=1)], z)

In [14]:
f(33)

array(34.0)

### Shared variables

In [15]:
from theano import shared

In [16]:
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

In [17]:
state.set_value(0)

In [18]:
accumulator(1)

array(0)

In [19]:
decrementor = function([inc], state, updates=[(state, state-inc)])

In [20]:
decrementor(1)

array(1)

In [21]:
# demonstrate use of `givens`
identity = function([inc], state, givens=[(state, inc)])

In [22]:
identity(5)

array(5, dtype=int32)

In [23]:
add1 = function([], state, updates=[(state, state+1)])

In [24]:
add1()

array(0)

### Random numbers

In [25]:
from theano.tensor.shared_randomstreams import RandomStreams

In [26]:
srng = RandomStreams(seed=123)
rv_u = srng.uniform((2,2))
rv_n = srng.normal((2,2))
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)
nearly_zeros = function([], rv_u + rv_u - 2 * rv_u)

In [27]:
f()

array([[ 0.72803009,  0.59044123],
       [ 0.23715077,  0.69958932]])

In [28]:
nearly_zeros()

array([[ 0.,  0.],
       [ 0.,  0.]])

### Monte Carlo $\pi$ estimation

In [29]:
from theano.ifelse import ifelse

In [30]:
m = 100000 # number of trials
rv_u = srng.uniform((m,))
r2 = rv_u**2 + rv_u**2
pi_est = function([], 4*T.mean(r2 < 1))

In [31]:
pi_est()

array(2.82804)

Would expect the above to work from the tutorial. Instead, seem to need:

In [32]:
m = 1000000 # number of trials
rv_u1 = srng.uniform((m,))
rv_u2 = srng.uniform((m,))
r2 = rv_u1**2 + rv_u2**2
pi_est = function([], 4*T.mean(r2 < 1))

In [33]:
pi_est()

array(3.1452199999999997)

### Logistic regression

In [34]:
from sklearn.datasets import *
from sklearn.cross_validation import train_test_split

In [35]:
data = load_iris()
Xtrain, Xtest, ytrain, ytest = (
    train_test_split(data['data'], np.where(data['target'] == 0, 0, 1)))

In [36]:
n, d = Xtrain.shape

In [37]:
alpha = 1e-3 # learning rate
lmbd = 0.05  # regularization

In [38]:
X = T.matrix('X')
y = T.vector('y')
w = shared(np.random.randn(d), name='w')
b = shared(0., name='b')

p = 1 / (1 + T.exp(-T.dot(X, w) - b))
pred = p > 0.5
xent = -y * T.log(p) - (1-y) * T.log(1-p) # cross-entropy loss
cost = xent.mean() + lmbd * (w**2).sum()
gw, gb = T.grad(cost, [w, b])

train = function(inputs=[X, y],
                 outputs=[pred, xent],
                 updates=[(w, w - alpha*gw), 
                          (b, b - alpha*gb)])

predict = function(inputs=[X], outputs=pred)

In [43]:
training_steps = 100

for i in range(training_steps):
    pred, err = train(Xtrain, ytrain)
    
(predict(Xtest) == ytest).mean()

1.0